In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
# client = MongoClient('mongodb://localhost:27017/')
db = client.db_cdiscount

In [2]:
import io
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data

In [3]:
import numpy as np

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import keras
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [5]:
first = 0
num_batch = 4
# last = 82
last = 5000

In [6]:
epochs = 10
batch_size = 64
num_classes = db.cat_encode.count()

In [7]:
def simple_model(epochs=epochs, num_classes=num_classes, lrate=0.01, momentum=0.9):
    
    # Convolutional Layer
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(40, (3,3), input_shape = (180,180,3), activation='relu'))

    # Pooling Layer
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

#     model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
model =simple_model(epochs=epochs, num_classes=num_classes)

In [9]:
# counter = 0

# category_id = db.train_example.find({}, {'category_id':1, "_id" : 0})
# category_id = [i['category_id'] for i in list(category_id)]

# cat = db.cat_encode.find({"cat" : category_id},{"cat" : 1.0, "_id" : 0})
# cat = list(cat)

# # category_id.pop
# print(cat)

In [10]:
# tmppic1=np.array([])
# tmppic2=np.array([])

In [11]:
import time

In [ ]:
start_time = time.time()
counter = 0
for j in range(0,num_batch-1):
# for j in range(0,1):
    
    a1 = int(list(np.linspace(first,last,num_batch))[j])
    an = int(list(np.linspace(first,last,num_batch))[j+1])
    
#     print(a1,an)
    
    cur = db.train.find({})[a1:an]
    
    m= an-a1
    
    X_batch = np.zeros((m,180,180,3))  
    Y_batch = np.zeros((m,num_classes), dtype=int)

#     X_batch = np.empty((180,180,3))
#     Y_batch = np.empty((num_classes))
    
#     print('X-Batch shape: ', X_batch.shape )
#     print('Y-Batch shape: ', Y_batch.shape )
    
    i = 0
#     pic_array
    while (cur.alive):
        idx = cur.next()
        category_id =idx['category_id'] 
    #     print(category_id)

        cat = db.cat_encode.find_one({ "cat" : (category_id)}, {"cat" : 1.0, "_id" : 0})['cat']
    #     print(cat)
        
#         picture = np.float32(imread(io.BytesIO(idx['imgs'][0]['picture']))/255.0)
        picture = np.float32(imread(io.BytesIO(idx['imgs'][0]['picture']))/255.0)
        picture = picture.reshape(180,180,3)
        
        X_batch[i,:,:,:] = picture
        
        
        encode = db.cat_encode.find_one({ "cat" : (category_id)}, {"encode" : 1.0, "_id" : 0})['encode']
    #     print(encode)
        Y_batch[i,:] = np.array(encode)[0]

        
        i=i + 1
    
#     print(Y_batch)
    print('encode shape', Y_batch.shape)
    print('Pic shape', X_batch.shape)
    
#     Y_batch1 = Y_batch
#     X_batch1 = X_batch
    
    model.fit(X_batch, Y_batch, batch_size=batch_size, epochs=epochs, validation_split=0.2)
    del(X_batch)
    del(Y_batch)
    
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# del(cur)

In [ ]:
# lst_batch

In [ ]:
start_time = time.time()
counter = 0
for j in range(0,num_batch-1):
# for j in range(0,1):
    
    a1 = int(list(np.linspace(first,last,num_batch))[j])
    an = int(list(np.linspace(first,last,num_batch))[j+1])
    
    lst_batch = []
    ## training Data
    for i in (db.train.find({})[a1:an]):
        dic = {}
    #     print('category_id: ', i['category_id'])
        dic['category_id'] = i['category_id']

    #     print('cat: ', db.cat_encode.find_one({ "cat" : (1000004079)}, {"cat" : 1.0, "_id" : 0})['cat'])
        dic['cat'] = db.cat_encode.find_one({ "cat" : (i['category_id'])}, {"cat" : 1.0, "_id" : 0})['cat']

    #     print('img: ', i['imgs'])
#         dic['imgs'] = i['imgs']
        picture = imread(io.BytesIO(i['imgs'][0]['picture']))
    ## divide image by 255.0
        picture = picture/255.0
        dic['picture'] = picture.reshape(180,180,3)

    #     print('encode: ', db.cat_encode.find_one({ "cat" : (1000004079)}, {"encode" : 1.0, "_id" : 0})['encode'])
        dic['encode'] = db.cat_encode.find_one({ "cat" : (i['category_id'])}, {"encode" : 1.0, "_id" : 0})['encode']

        lst_batch.append(dic)
             
        counter = counter +1
        
    
    X_batch = np.array([lst['picture'] for lst in lst_batch])
    Y_batch = np.array([lst['encode'] for lst in lst_batch])
#     print(X_batch.shape)
#     print(Y_batch.shape)
    
#     print(Y_batch)
    print(X_batch.shape)
    print(Y_batch.shape)
    
    
#     Y_batch2 = Y_batch
#     X_batch2 = X_batch

    
    model.fit(X_batch, Y_batch, batch_size=batch_size, epochs=epochs, validation_split=0.2)
    #, validation_split=0.2
print(an)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# X_batch1[0]

In [ ]:
# X_batch2[0]

In [ ]:
# print(X_batch1.shape)
# print(X_batch2.shape)
# # print(np.equal(X_batch1[0], X_batch2[0]))
# t = 0
# f = 0
# for j in (np.equal(X_batch1[0], X_batch2[0])):
#     print(j)
# #     if(i is True):
# #         t = t+1
# #     if(i is False):
# #         f = f+1
        
# print(t,f)

In [ ]:
# len(np.unique([cat['category_id'] for cat in lst_batch]))

In [ ]:
# num_classes=len(dummy_y[81])
# epochs = 5

In [12]:
start_time = time.time()

counter = 0
for j in range(0,num_batch-1):
# for j in range(0,1):
    
    a1 = int(list(np.linspace(first,last,num_batch))[j])
    an = int(list(np.linspace(first,last,num_batch))[j+1])
    
    lst_batch = []
    
#     print(a1,an)
    
    cur = db.train.find({})[a1:an]
    
    m= an-a1
        
    i = 0
#     pic_array
    while (cur.alive):
        idx = cur.next()
        dic = {}
        
        category_id =idx['category_id'] 
    #     print(category_id)

        cat = db.cat_encode.find_one({ "cat" : (category_id)}, {"cat" : 1.0, "_id" : 0})['cat']
    #     print(cat)
        
        picture = np.float32(imread(io.BytesIO(idx['imgs'][0]['picture']))/255.0)
        picture = picture.reshape(180,180,3)
        
        dic['picture'] = picture
        
        encode = db.cat_encode.find_one({ "cat" : (category_id)}, {"encode" : 1.0, "_id" : 0})['encode']
    #     print(encode)
        dic['encode'] = encode
        
        lst_batch.append(dic)
        
        i+=1
    X_batch = np.array([lst['picture'] for lst in lst_batch])
    Y_batch = np.array([lst['encode'] for lst in lst_batch])
    print(X_batch.shape)
    print(Y_batch.shape)
    

    
    model.fit(X_batch, Y_batch, batch_size=batch_size, epochs=epochs, validation_split=0.2)
    
print("--- %s seconds ---" % (time.time() - start_time))

(1666, 180, 180, 3)
(1666, 5270)
Train on 1332 samples, validate on 334 samples
Epoch 1/10
1332/1332 [==============================] - 12s - loss: 5.2992 - acc: 0.3994 - val_loss: 4.4043 - val_acc: 0.4311
Epoch 2/10
1332/1332 [==============================] - 9s - loss: 3.6737 - acc: 0.4257 - val_loss: 4.3837 - val_acc: 0.4311
Epoch 3/10
1332/1332 [==============================] - 8s - loss: 3.2098 - acc: 0.4377 - val_loss: 4.3746 - val_acc: 0.4491
Epoch 4/10
1332/1332 [==============================] - 8s - loss: 2.8597 - acc: 0.4527 - val_loss: 4.1703 - val_acc: 0.4611
Epoch 5/10
1332/1332 [==============================] - 8s - loss: 2.5288 - acc: 0.4797 - val_loss: 4.2854 - val_acc: 0.4581
Epoch 6/10
1332/1332 [==============================] - 8s - loss: 2.1516 - acc: 0.5105 - val_loss: 4.4207 - val_acc: 0.4671
Epoch 7/10
1332/1332 [==============================] - 8s - loss: 1.9070 - acc: 0.5480 - val_loss: 4.6171 - val_acc: 0.4641
Epoch 8/10
1332/1332 [======================

KeyboardInterrupt: 